In [1]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
from os.path import join as oj
from copy import deepcopy
import pandas as pd
from numpy import array as arr
import time

# sklearn models
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import export_graphviz, DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.tree import plot_tree

import torch
from torch import nn
from util import t, to_sparse

import predict
from rf_to_dnn import Net

In [ ]:
def f1(): rf.predict(X)
def f2(): predict.f_torch_basic(net, X_t)
def f3(): predict.f_torch_basic(netg, X_tg)
def f4(): predict.f_np_basic(X, idxs0, b0, w1, b1, idxs2)
def f5(): predict.f_torch_indexing(X_t, idxs0, b0_t, lay1, b1_t, idxs2_t)

np.random.seed(13)

max_depths = []
ts = []
# for N in [10, 100, 1000, 10000, 100000]:
for N in [100]:
    num_features = 4
#     N = 1000
    max_depth = 100

    # prepare data
    X = np.random.rand(N, num_features)
    y = np.random.rand(N)
    X_t = torch.Tensor(X)
    X_tg = deepcopy(torch.Tensor(X)).to('cuda')

    rf = DecisionTreeRegressor(max_depth=max_depth)
    rf.fit(X, y)
    print(N, 'max depth', rf.get_depth())
    max_depths.append(rf.get_depth())

    # plt.scatter(X[:, 0], X[:, 1], c=rf.predict(X))
    # plt.show()
    # plot_tree(rf.estimators_[0])
    # plot_tree(rf)
    # plt.show()

    # pepare net
    net = Net(rf)
    b0, idxs0, w1, b1, idxs2 = net.extract_util_np()
    idxs0_t, b0_t, lay1, b1_t, idxs2_t = \
        torch.Tensor(idxs0), torch.Tensor(b0), net.layers[1], \
        torch.Tensor(b1), torch.Tensor(idxs2)
    netg = deepcopy(net).to('cuda')
    net_s = deepcopy(net)
    net_s.layers[1].weight.data = to_sparse(net.layers[1].weight)
    # w1 = sparse.csr_matrix(w1)
    
    
    timings = {}
    for (f, s) in zip([f1, f2, f3, f4, f5], 
                      ['rf', 't_basic', 't gpu', 'np_basic', 't_ind']):
        with torch.no_grad():
            timings[s] = t(f)
            print(f'\t{s}\t{timings[s]:0.2e}')
    ts.append(timings)

In [ ]:
if np.isclose(rf.predict(X).flatten(), p3.flatten()).all():
    print('success!')
else:
    print('fail')